In [ ]:
import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Getting the data

In [ ]:
dataset = pd.read_pickle('data/datasets/dataset_with_features.pickle')

In [ ]:
# Selecting the numerical features created
features = dataset[['max_sentiment_english',
                    'max_sentiment_german',
                    'std_max_english_sentiment',
                    'std_max_german_sentiment',
                    'german_sentence_length',
                    'english_sentence_length',
                    'sentence_length_difference',
                    'verbs_diff',
                    'adjectives_diff',
                    'adverbs_diff',
                    'nouns_diff',
                    'non_translated_words',
                    'correlation',
                    'sentence_correlation',
                    'non_match_correlation',
                    'non_translated_ratio']]

In [ ]:
# Splitting between the Train/Validation dataset and the test
train_val = features[:8000]
test = features[8000:]

In [ ]:
# Setting the features to a list (used for random forest important features)
dataset_features_list = list(train_val.columns)
dataset_features_arr = np.array(train_val)
test_features_arr = np.array(test)

In [ ]:
# Collecting the scores for the train and validation datasets
path_train_scores = os.path.join(os.getcwd(), 'data', 'en-de', 'train.ende.scores')
train_scores = pd.read_csv(path_train_scores,header=None)
train_scores = train_scores.rename(columns={0:"scores"})
path_val_scores = os.path.join(os.getcwd(), 'data', 'en-de', 'dev.ende.scores')
val_scores = pd.read_csv(path_val_scores,header=None)
val_scores = val_scores.rename(columns={0:"scores"})
scores = pd.concat([train_scores, val_scores])

dataset_labels_arr = np.array(scores)

## CROSS VALIDATION AND RESULTS

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV

In [ ]:
# Lines to standardize the features if desired

# normalised = (dataset_features_arr - dataset_features_arr.min(axis=0))/ \
#              (dataset_features_arr.max(axis=0) - dataset_features_arr.min(axis=0))
# normalised.var(axis=0)
# sel = VarianceThreshold(threshold=0.01)
# dataset_features_arr = sel.fit_transform(normalised)

### PLS

In [ ]:
from sklearn.cross_decomposition import PLSRegression

In [ ]:
# Performing a randomized grid search and cross validation to find the best PLSRegressor model 

n_components = [int(x) for x in np.arange(1,12)]
scale = [True, False]
max_iter = [int(x) for x in np.arange(300,1000)]

random_grid = {'n_components': n_components,
               'scale': scale, 
               'max_iter':max_iter}

pls = PLSRegression()
pls_random = RandomizedSearchCV(estimator = pls, param_distributions = random_grid, 
                                n_iter = 100, cv = 8, random_state=42, scoring='neg_mean_squared_error')
pls_random.fit(dataset_features_arr, dataset_labels_arr)
print(pls_random.best_params_)

#### In the next cell, a cross validation over many randomly created splits are run on each of the best model to provide estimates of the score distribution. 

In [ ]:
rs = ShuffleSplit(n_splits=500, test_size=.25, random_state=0)
cv = rs.split(dataset_features_arr)

correlation_pls = []
mae_pls = []
X = dataset_features_arr
y = dataset_labels_arr.reshape(-1)

for train, test in cv:
    pls = PLSRegression(**pls_random.best_params_)
    pls.fit(X[train], y[train])
    predictions = pls.predict(X[test]).reshape(-1)
    pearson = pearsonr(y[test], predictions)[0]
    error = abs(predictions - y[test])
    correlation_pls.append(pearson)
    mae_pls.append(error)
    
print(np.array(correlation_pls).mean())
print(np.array(mae_pls).mean())
print(np.array(correlation_pls).std())

In [ ]:
# Create predictions directory if don't exist
predictions_dir = os.path.join(os.getcwd(), 'data', 'imgs')
if not os.path.exists(predictions_dir):
    os.mkdir(predictions_dir)

In [ ]:
sns.distplot(correlation_pls)
plt.xlabel('Pearson Correlation')
plt.ylabel('Counts')
plt.savefig('data/imgs/pls.svg', bbox_inches='tight')

### Linear Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Performing a randomized grid search and cross validation to find the best Ridge Linear Regression model 

alpha = [x for x in np.linspace(0,1,10)]
normalize = [True, False]
max_iter = [int(x) for x in np.arange(300,1500)]

random_grid = {'alpha': alpha,
               'normalize': normalize, 
               'max_iter':max_iter}

lr = Ridge()
lr_random = RandomizedSearchCV(estimator = lr, param_distributions = random_grid, 
                                n_iter = 100, cv = 8, random_state=42, scoring='neg_mean_squared_error')
lr_random.fit(dataset_features_arr, dataset_labels_arr)
print(lr_random.best_params_)

#### In the next cell, a cross validation over many randomly created splits are run on each of the best model to provide estimates of the score distribution. 

In [ ]:
# Making a cross validation over 500 different splits to compute an estimate of the score distribution

rs = ShuffleSplit(n_splits=500, test_size=.25, random_state=0)
cv = rs.split(dataset_features_arr)

correlation_ridge = []
mae_ridge = []
X = dataset_features_arr
y = dataset_labels_arr.reshape(-1)

# for all different split
for train, test in cv:
    rl = Ridge(**lr_random.best_params_)
    rl.fit(X[train], y[train])
    predictions = rl.predict(X[test]).reshape(-1)
    pearson = pearsonr(y[test], predictions)[0]
    error = abs(predictions - y[test])
    correlation_ridge.append(pearson)
    mae_ridge.append(error)

# Print the score of interest
print(np.array(correlation_ridge).mean())
print(np.array(mae_ridge).mean())
print(np.array(correlation_ridge).std())

In [ ]:
# Plot and save the distribution
sns.distplot(correlation_ridge)
plt.xlabel('Pearson Correlation')
plt.ylabel('Counts')
plt.savefig('data/imgs/ridge.svg', bbox_inches='tight')

### SVM

In [ ]:
from sklearn.svm import SVR

In [ ]:
# Performing a randomized grid search and cross validatino to find the best SVM model 

kernel = ('linear', 'poly', 'rbf')
C = [int(x) for x in np.linspace(1, 150, 10)]

random_grid = {'kernel': kernel,
               'C': C}
svm = SVR()
svm_random = RandomizedSearchCV(estimator = svm, param_distributions = random_grid, 
                                n_iter = 10, cv = 2, scoring='neg_mean_squared_error')
svm_random.fit(dataset_features_arr, dataset_labels_arr.ravel())
print(svm_random.best_params_)

#### In the next cell, a cross validation over many randomly created splits are run on each of the best model to provide estimates of the score distribution. 

In [ ]:
rs = ShuffleSplit(n_splits=10, test_size=.25, random_state=0)
cv = rs.split(dataset_features_arr)

correlation_svm = []
mae_svm = []
X = dataset_features_arr
y = dataset_labels_arr.reshape(-1)

for train, test in cv
    svm = SVR(**svm_random.best_params_)
    svm.fit(X[train], y[train])
    predictions = svm.predict(X[test]).reshape(-1)
    pearson = pearsonr(y[test], predictions)[0]
    error = abs(predictions - y[test])
    correlation_svm.append(pearson)
    mae_svm.append(error)
    
print(np.array(correlation_svm).mean())
print(np.array(mae_svm).mean())
print(np.array(correlation_svm).std())

sns.distplot(correlation_svm)
plt.xlabel('Pearson Correlation')
plt.ylabel('Counts')
plt.savefig('data/imgs/svr.svg', bbox_inches='tight')

### Random Forests

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Performing a randomized grid search and cross validation to find the best random forest model 
# Setting the different parameters

n_estimators = [500, 1000]
max_depth = [1,2]
bootstrap = [True]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'bootstrap':bootstrap}

rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 2, cv = 2, random_state=42, scoring='neg_mean_squared_error')
rf_random.fit(dataset_features_arr, dataset_labels_arr.ravel())
print(rf_random.best_params_)
print(rf_random.best_score_)

In [ ]:
# Highlighting the most important features of the model
rf = RandomForestRegressor(**{'n_estimators': 1000, 'max_depth': 2, 'bootstrap': True})
rf.fit(dataset_features_arr, dataset_labels_arr.ravel())

importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(dataset_features_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

#### In the next cell, a cross validation over many randomly created splits are run on each of the best model to provide estimates of the score distribution. 

In [ ]:
rs = ShuffleSplit(n_splits=10, test_size=.25, random_state=0)
cv = rs.split(dataset_features_arr)

correlation_rf = []
mae_rf = []
X = dataset_features_arr
y = dataset_labels_arr.reshape(-1)

for train, test in cv
    rf = RandomForestRegressor(**rf_random.best_params_)
    rf.fit(X[train], y[train])
    predictions = rf.predict(X[test]).reshape(-1)
    pearson = pearsonr(y[test], predictions)[0]
    error = abs(predictions - y[test])
    correlation_rf.append(pearson)
    mae_rf.append(error)
    
print(np.array(correlation_rf).mean())
print(np.array(mae_rf).mean())
print(np.array(correlation_rf).std())

sns.distplot(correlation_rf)
plt.xlabel('Pearson Correlation')
plt.ylabel('Counts')
plt.savefig('data/imgs/rf.svg', bbox_inches='tight')

# Nerual Network

The final model tested were neural networks. The following provides the implementation of the network using Pytorch, as well as a cross validation on the tested architecture. In this section, the full LASER embedding 1024 dimension vectors were included in the feartures.

In [ ]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
import torch.optim as optim

In [ ]:
# Load the laser embeddings daatsets
english_laser = np.load('data/laser_embeddings/laser_1024_english.npy')
german_laser = np.load('data/laser_embeddings/laser_1024_german.npy')

In [ ]:
# Select the train and validation features
english_laser_train = english_laser[:8000,:]
german_laser_train = german_laser[:8000,:]

In [ ]:
# Get arrays from pandas columns
english_laser_train = np.array(english_laser_train)
german_laser_train = np.array(german_laser_train)

In [ ]:
# Get the total features
laser_features = np.concatenate((english_laser_train, german_laser_train), axis=1)
train_val_features = np.concatenate((laser_features, dataset_features_arr), axis = 1)

In [ ]:
# Constructing the model

class Net(nn.Module):

    def __init__(self, x):
        super(Net, self).__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(x.shape[1], 1000) 
        nn.init.xavier_uniform_(self.fc1.weight, gain=1.0)
        self.fc2 = nn.Linear(1000, 500)
        nn.init.xavier_uniform_(self.fc2.weight, gain=1.0)
        self.fc3 = nn.Linear(500, 1)
        nn.init.xavier_uniform_(self.fc2.weight, gain=1.0)

    def forward(self, x):
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net(train_val_features)
print(net)

In [ ]:
# Performing cross validation over random splits of the data, and printing the best pearson correlation 
# coefficents obtained during on the test data

rs = ShuffleSplit(n_splits=5, test_size=.25, random_state=0)
cv = rs.split(dataset_features_arr)

for train_idx, val_idx in cv:
    
    # Rededining the model to restars learning
    net = Net(train_val_features)
    
    train_features = train_val_features[train_idx]
    train_labels = dataset_labels_arr[train_idx].reshape(-1)
    
    val_features = train_val_features[val_idx]
    val_labels = dataset_labels_arr[val_idx].reshape(-1)
    
    # Making to tensors as required by Pytorch to propagate the gradients
    train_features_ten, train_labels_ten = Tensor(train_features), Tensor(train_labels.reshape(-1,1))

    # Standardizing the inputs
    means = train_features_ten.mean(dim=0, keepdim=True)
    stds = train_features_ten.std(dim=0, keepdim=True)
    normalized_train = (train_features_ten - means) / stds

    val_features_ten = (Tensor(val_features)- means) / stds
    test_labels = Tensor((val_labels).reshape(-1, 1))

    # Creating the dataloaders to create the batches and iterate over epochs
    train_data = TensorDataset(normalized_train, train_labels_ten)
    train_loader = DataLoader(train_data, shuffle=True, batch_size=32)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters(), lr=1e-4, betas=(0.5, 0.999))

    test_pearson=[]
    train_pearson=[]
    
    for epoch in range(15):
        running_loss = 0.0
        for batch_idx, data in enumerate(train_loader):
            inputs, labels = data
            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()

        # No grads to prevent from propagating the gradient from the test data
        with torch.no_grad():
            # Getting the test correlation
            predictions = net(val_features_ten)
            test_loss = criterion(predictions, test_labels).item()
            pearson_test = pearsonr(val_labels, predictions.numpy().reshape(-1,))[0]
            test_pearson.append(pearson_test)

            #train correlation
            predictions = net(normalized_train)
            pearson_train = pearsonr(train_labels, predictions.numpy().reshape(-1,))[0]
            train_pearson.append(pearson_train)

    print('Test: ', test_pearson.index(max(test_pearson)), max(test_pearson))
    print('Train: ', train_pearson.index(max(train_pearson)), max(train_pearson))
    plt.plot(test_pearson)
    print('Finished Training')

## Predictions for test set with the best set of parameters for the Ridge and PLS

In [ ]:
# Make predictions with ridge regression 

lr_test = Ridge(**{'normalize': True, 'max_iter': 595, 'alpha': 0.2222222222222222})
lr_test.fit(dataset_features_arr, dataset_labels_arr)
predictions = lr_test.predict(test_features_arr).reshape(-1,)

In [ ]:
# Making predictions with PLS regression 

pls_test = PLSRegression(**{'scale': True, 'n_components': 2, 'max_iter': 339})
pls_test.fit(dataset_features_arr, dataset_labels_arr)
predictions = pls_test.predict(test_features_arr).reshape(-1,)

In [ ]:
# Create predictions directory if don't exist
predictions_dir = os.path.join(os.getcwd(), 'data', 'predictions')
if not os.path.exists(predictions_dir):
    os.mkdir(predictions_dir)

In [ ]:
# Writing the scores to be tested on Codalab

def writeScores(method_name,scores):
    fn = "data/predictions/predictions.txt"
    print("")
    with open(fn, 'w') as output_file:
        for idx,x in enumerate(scores):
            #out =  metrics[idx]+":"+str("{0:.2f}".format(x))+"\n"
            #print(out)
            output_file.write(f"{x}\n")

writeScores("model_name",predictions)